In [1]:
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/train.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/valid.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/test.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [2]:
# Import OS module
import os

# Imports
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt

# Tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import callbacks, layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img,img_to_array
from keras.applications import imagenet_utils
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Sequential
from keras.applications.vgg19 import VGG19 , preprocess_input , decode_predictions
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Notebook magic
%matplotlib inline

In [3]:
# Configure variables for Transfer learning
image_size = 224
target_size = (image_size, image_size)
input_shape = (image_size, image_size, 3)
grid_shape = (1, image_size, image_size, 3)

batch_size = 32

In [4]:
train_dir = "/content/train"
test_dir = "/content/test"
valid_dir = "/content/valid"

In [ ]:
# Define augmentations for train dataset and read the images
train_aug = ImageDataGenerator(
    # Rescale
    rescale=1/255.0,
    # Filling for W/H shift
    fill_mode="nearest",
    # Width and Height shift
    width_shift_range=0.2,
    height_shift_range=0.2,
    # Random zooms
    zoom_range=0.2,
    # Random Shearing aug
    shear_range=0.2,
)

# Read data from directory
train_data = train_aug.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical"
)

In [ ]:
cats = list(train_data.class_indices.keys())

In [11]:
# Augmentations for test data
test_aug = ImageDataGenerator(
    # Rescale
    rescale=1/255.0
)

# Read data from directory
test_data = test_aug.flow_from_directory(
    test_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical"
)

valid_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

# Read data from directory
valid_data = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical"
)

Found 33 images belonging to 1 classes.
Found 17572 images belonging to 38 classes.


In [12]:
train_class = train_data.classes
test_class = test_data.classes
valid_class = valid_data.class_indices

**MobileNet V2**

In [13]:
# Load the base model
mbnet_v2 = keras.applications.MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=input_shape
)

# Stop from being trainable
mbnet_v2.trainable = False

9406464/9406464 [==============================] - 0s 0us/step


In [14]:
# Define the layers
inputs = keras.Input(shape=input_shape)

# Get the layer
x = mbnet_v2(inputs, training = False)

# Stack layers further
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(len(cats), activation="softmax")(x)

# Combine the model
model = Model(inputs=inputs, outputs=x)

# Summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 38)                48678     
                                                                 
Total params: 2306662 (8.80 MB)
Trainable params: 48678 (190.

In [15]:
# Compile
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# Num epochs
epochs = 30

# Train model
history = model.fit(
    train_data,
    epochs=epochs,
    steps_per_epoch=200,
    validation_data=valid_data,
)

Epoch 1/30
 97/200 [=============>................] - ETA: 2:48 - loss: 2.4036 - accuracy: 0.3785

In [ ]:
evaluate = model.evaluate(valid_data)
print(evaluate)

In [ ]:
hist = history.history
# Plot accuracy and loss
plt.plot(hist["accuracy"], label="accuracy")
plt.plot(hist["val_accuracy"], label="val_accuracy")

# Add the labels and legend
plt.ylabel("Accuracy / Loss")
plt.xlabel("Epochs #")
plt.legend()

plt.show()

In [ ]:
if "val_accuracy" in hist and "val_loss" in hist:
    plt.plot(hist["loss"], label="loss")
    plt.plot(hist["val_loss"], label="val_loss")

# Add the labels and legend
plt.ylabel("Accuracy / Loss")
plt.xlabel("Epochs #")
plt.legend()

plt.show()

In [ ]:
model.save("plant_disease_detection.h5")

In [ ]:
import json

from IPython.display import FileLink

# Link the files
FileLink("./plant_disease_detection.h5")

In [ ]:
# Dump the categories
with open("categories.json", "w") as file:
  json.dump(train_data.class_indices, file)

# Link categories JSON
FileLink("categories.json")

In [ ]:
class_mapping = {
    0: 'Apple Scab',
    1: 'Apple Black Rot',
    2: 'Apple Rust',
    3: 'Apple Healthy',
    4: 'Blueberry Healthy',
    5: 'Cherry (including sour) Powdery mildew',
    6: 'Cherry (including sour) healthy',
    7: 'Corn (maize) Cercospora leaf spot Gray leaf spot',
    8: 'Corn (maize) Common rust',
    9: 'Corn (maize) Northern Leaf Blight',
    10: 'Corn (maize) Healthy',
    11: 'Grape Black Rot',
    12: 'Grape Esca (Black Measles)',
    13: 'Grape Leaf blight (Isariopsis Leaf Spot)',
    14: 'Grape Healthy',
    15: 'Orange Haunglongbing (Citrus greening)',
    16: 'Peach Bacterial Spot',
    17: 'Peach Healthy',
    18: 'Pepper Bell Bacterial Spot',
    19: 'Pepper Bell Healthy',
    20: 'Potato Early Blight',
    21: 'Potato Late Blight',
    22: 'Potato Healthy',
    23: 'Raspberry Healthy',
    24: 'Soyabean Healthy',
    25: 'Squash Powdery Mildew',
    26: 'Strawberry Leaf Scorch',
    27: 'Strawberry Healthy',
    28: 'Tomato Bacterial Spot',
    29: 'Tomato Early Blight',
    30: 'Tomato Late Blight',
    31: 'Tomato Leaf Mold',
    32: 'Tomato Septoria leaf spot',
    33: 'Tomato Spider mites Two-spotted spider mite',
    34: 'Tomato Target Spot',
    35: 'Tomato Tomato Yellow Leaf Curl Virus',
    36: 'Tomato Tomato mosaic virus',
    37: 'Tomato Healthy',
    38: 'None',
    }

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

image_path = '/content/valid/Cherry_(including_sour)___Powdery_mildew/02ecd449-7b27-44d7-b59e-7e31813f0c18___FREC_Pwd.M 0450.JPG'
img = image.load_img(image_path, target_size=(224, 224))  # Adjust target_size as needed
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)


In [ ]:
predictions = model.predict(img_array)

In [ ]:
predicted_class_index = np.argmax(predictions)
print(predicted_class_index)
predicted_disease_class = class_mapping.get(predicted_class_index, 'Unknown')
print("Predicted Disease Class:", predicted_disease_class)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
y_pred = model.predict(valid_data, steps=len(valid_data), verbose=1)

In [ ]:
class_indices = valid_data.class_indices

In [ ]:
y_true = [class_indices[filename.split('/')[0]] for filename in valid_data.filenames]

In [ ]:
report = classification_report(y_true, np.argmax(y_pred, axis=1))
print("Classification Report:")
print(report)